# SVD 奇异值分解
奇异值分解(Singular Value Decomposition：
> 优点：简化数据，去除噪声，提高算法运行效果<br>
> 缺点：数据的转换可能难以理解<br>
> 适用数据类型：数值型数据

假设矩阵A是m*n的，奇异值分解的结果就是将矩阵A分解为如下三个矩阵的乘积：
$$A_{mxn} = U_{mxm}\Sigma_{mxn}{V^T_{nxn}}$$
U 的列由 $AA^T$ 的单位化过的特征向量构成<br>
V 的列由 $A^TA$ 的单位化过的特征向量构成<br>
Σ 的对角元素来源于 $AA^T$ 或 $A^TA$ 的特征值的平方根，并且是按从大到小的顺序排列的。奇异值就是 Σ 对角线上的元素。

## 1. 使用Python实现SVD

In [1]:
import numpy as np

In [4]:
U, sigma, VT = np.linalg.svd([[1, 1], [7, 7]])
print(U, '\n\n', sigma, '\n\n', VT)

[[-0.14142136 -0.98994949]
 [-0.98994949  0.14142136]] 

 [  1.00000000e+01   2.82797782e-16] 

 [[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]


In [5]:
def loadExData():
    return [[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]]

In [7]:
Data = loadExData()
U, Sigma, VT = np.linalg.svd(Data)
Sigma

array([  9.72140007e+00,   5.29397912e+00,   6.84226362e-01,
         3.19320674e-16,   3.20843997e-32])

可以看到前三个数值比后面的大了很多，近似可以把后面两个数据看成0.

In [8]:
# 重构原矩阵
Sig3 = np.mat([[Sigma[0], 0, 0], [0, Sigma[1], 0], [0, 0, Sigma[2]]])
U[:, :3] * Sig3 * VT[:3, :]

matrix([[  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
           3.61262144e-16,   3.70225958e-16],
        [  2.00000000e+00,   2.00000000e+00,   2.00000000e+00,
          -1.46415982e-16,  -1.28488353e-16],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -1.33036301e-15,  -1.32139919e-15],
        [  5.00000000e+00,   5.00000000e+00,   5.00000000e+00,
           1.93978525e-16,   2.38689177e-16],
        [  1.00000000e+00,   1.00000000e+00,  -1.42782680e-15,
           2.00000000e+00,   2.00000000e+00],
        [  3.78943636e-16,   2.20678054e-16,  -5.66590640e-16,
           3.00000000e+00,   3.00000000e+00],
        [  1.15199728e-16,   6.70704628e-17,  -1.98958284e-16,
           1.00000000e+00,   1.00000000e+00]])

## 2. 基于协同过滤的推荐引擎
### a. 相似度计算
欧氏距离、皮尔逊相似度、余弦相似度

In [9]:
# 计算欧氏距离
def  ecludSim(inA, inB):
    return 1.0 / (1.0 + np.linalg.norm(inA - inB))

# 计算皮尔逊距离
def pearsSim(inA, inB):
    # 如果不存在，则返回1.0，这时完全相关
    if len(inA) < 3:
        return 1.0
    return 0.5 + 0.5 * np.corrcoef(inA, inB, rowvar=0)[0][1]

# 计算余弦相似度，夹角为90相似度为0.
def cosSim(inA, inB):
    num = float(inA.T * inB)
    denom = np.linalg.norm(inA) * np.linalg.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [10]:
myMat = np.mat(loadExData())
ecludSim(myMat[:, 0], myMat[:, 4])

0.13367660240019172

In [11]:
ecludSim(myMat[:, 0], myMat[:, 0])

1.0

In [13]:
pearsSim(myMat[:, 0], myMat[:, 4]), pearsSim(myMat[:, 0], myMat[:, 1])

(0.23768619407595815, 1.0)

### b. 餐馆菜肴推荐引擎
推荐系统的工作过程是:给定一个用户,系统会为此用户返回N个最好的推荐菜。为了实现这一点,则需要我们做到:
    1. 寻找用户没有评级的菜肴,即在用户-物品矩阵中的 0 值;
    2. 在用户没有评级的所有物品中,对每个物品预计一个可能的评级分数。这就是说,我们认为用户可能会对物品的打分(这就是相似度计算的初衷);
    3. 对这些物品的评分从高到低进行排序,返回前N个物品。

In [14]:
def standEst(dataMat, user, simMeas, item):
    n = dataMat.shape[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        overLap = np.nonzero(np.logical_and(dataMat[:, item].A > 0,
                                            dataMat[:, j].A > 0))[0]
        if len(overLap) == 0:
            similarity = 0
        else:
            similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        print("The %d and %d similarity is: %f" % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal / simTotal
    
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = np.nonzero(dataMat[user, :].A == 0)[1]
    if len(unratedItems) == 0:
        return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]